<a href='http://www.holoviews.org'><img src="assets/header_logo.png" alt="HoloViews logo" width="20%;" align="left"/></a>
<div style="float:right;"><h2>08. Deploying Bokeh Apps</h2></div>

In [ ]:
import geoviews as gv
import dask.dataframe as dd
from holoviews.operation.datashader import datashade
import holoviews as hv

hv.extension('bokeh')

## Loading data

As before we will load the taxi dataset as a dask dataframe:

In [ ]:
usecols = ['tpep_pickup_datetime', 'dropoff_x', 'dropoff_y']
ddf = dd.read_csv('../data/nyc_taxi.csv', parse_dates=['tpep_pickup_datetime'], usecols=usecols)
ddf['hour'] = ddf.tpep_pickup_datetime.dt.hour
ddf = ddf.persist()

## Declare the visualization

We will also once again declare a tile source and datashade the dropoff locations:

In [ ]:
from bokeh.models import WMTSTileSource
url = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg'
wmts = gv.WMTS(WMTSTileSource(url=url))

points = hv.Points(ddf, kdims=['dropoff_x', 'dropoff_y'])
shaded = datashade(points).opts(plot=dict(width=800, height=400, xaxis=None, yaxis=None))

## Deploy app in notebook

When working within the Jupyter notebook we can easily start a server using the HoloViews ``BokehRenderer``. We get a handle on the renderer using the ``hv.renderer`` function. Now we can use the ``app`` method passing it our HoloViews object and setting ``show=True``. If your notebook server is not running on the default port (8888), you also need to define the ``websocket_origin``:

In [ ]:
renderer = hv.renderer('bokeh')
server = renderer.app(wmts * shaded, show=True, websocket_origin='localhost:8890')

#### Excercise 1: Launch a bokeh app using bokeh serve

1. Open http://localhost:8888/edit/notebooks/server_app.py
2. Open http://localhost:8888/terminals/1
3. Now execute:

```
source activate hvtutorial
cd apps
bokeh serve --show server_app.py
```

## Adding functionality

Now let us add some custom functionality to the app we just created. We will declare a ``Stream`` that allows selecting the hour of the day and then declare a DynamicMap that applies the selection.

In [ ]:
from bokeh.models import WMTSTileSource
url = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg'
wmts = gv.WMTS(WMTSTileSource(url=url))

stream = hv.streams.Stream.define('HourSelect', hour=0)()
dmap = hv.DynamicMap(lambda hour: points.select(hour=hour), streams=[stream])
shaded = datashade(dmap)

hv.opts('RGB [width=800 height=400 xaxis=None yaxis=None]')

hvobj = wmts * shaded
hvobj

We can now manually trigger updates using the ``stream.event`` method:

In [ ]:
stream.event(hour=6)

In a bokeh server app we can link this stream to another widget, which is exactly what we will do now:

#### Exercise 2: Modify app

1. Open http://localhost:8888/edit/notebooks/player_app.py
2. Copy the HoloViews example above into the appropriate place in the player_app script and note where the ``stream.event`` method is called
2. Open http://localhost:8888/terminals/1
3. Now shut down the server we created in exercise 1 
4. Now execute:

```
bokeh serve --show player_app.py
```